In [58]:
import pandas as pd
import re

# Clean input into dataframe

In [59]:
raw_input = pd.read_excel("DATA/Day_02_Possible_Game.xlsx")

In [60]:
raw_input[['Game', 'Sets']] = raw_input['Input'].str.split(": ", n = 1, expand=True)

In [61]:
columns = raw_input.Sets.max().split(';')

new = raw_input.Sets.str.split(";", expand = True)
new.columns = columns
new = new.add_prefix("Set")

raw_input = raw_input.join(new).drop("Sets", 1)
raw_input = raw_input.rename(columns = {
   "Set9 red": "Set_1",
   "Set 10 red, 2 blue": "Set_2",
   "Set 3 red": "Set_3",
   "Set 8 red, 1 green, 2 blue": "Set_4",
   "Set 1 red, 2 blue": "Set_5",
   "Set 1 blue, 4 red" : "Set_6"
})
raw_input = raw_input.drop(["Input"], 1)

/tmp/ipykernel_603451/3699298453.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  raw_input = raw_input.join(new).drop("Sets", 1)
/tmp/ipykernel_603451/3699298453.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  raw_input = raw_input.drop(["Input"], 1)


In [62]:
raw_input_long = raw_input.melt(id_vars="Game")
raw_input_long = raw_input_long[~raw_input_long['value'].isna()]
# raw_input_long.value = raw_input_long.value.fillna('')

In [63]:
def search_color_blue(set_string):

    rule_search = re.compile(r'([0-9]+)\sblue')

    color = rule_search.search(set_string)

    if color is None:

        return 0

    else:

        return color.group(1)

def search_color_red(set_string):

    rule_search = re.compile(r'([0-9]+)\sred')

    color = rule_search.search(set_string)

    if color is None:

        return 0

    else:

        return color.group(1)

def search_color_green(set_string):

    rule_search = re.compile(r'([0-9]+)\sgreen')

    color = rule_search.search(set_string)

    if color is None:

        return 0

    else:

        return color.group(1)

In [64]:
raw_input_long['Blue'] = raw_input_long.value.apply(search_color_blue)
raw_input_long['Red'] = raw_input_long.value.apply(search_color_red)
raw_input_long['Green'] = raw_input_long.value.apply(search_color_green)

In [65]:
raw_input_long = raw_input_long.drop(['value'], 1).rename(columns={
    'variable': 'Set'
})

/tmp/ipykernel_603451/2628947315.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  raw_input_long = raw_input_long.drop(['value'], 1).rename(columns={


In [66]:
input_data = raw_input_long.melt(id_vars=['Game', 'Set'],
                                 var_name='Color',
                                 value_name="Number")

In [67]:
input_data['Number'] = input_data.Number.astype(int)
input_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1359 entries, 0 to 1358
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Game    1359 non-null   object
 1   Set     1359 non-null   object
 2   Color   1359 non-null   object
 3   Number  1359 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 42.6+ KB


# Part 1: Check possible game

In [68]:
constraint_df = pd.DataFrame({
    'Color': ['Red', 'Green', 'Blue'],
    'Limit': [12, 13, 14]
})

In [69]:
constraint_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Color   3 non-null      object
 1   Limit   3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


In [70]:
check_df = input_data.merge(constraint_df, on='Color')
check_df['NotPossible'] = check_df.apply(
    lambda x: True if x.Number > x.Limit else False, axis=1
)

check_df = check_df.groupby(
    ['Game'],
    as_index=False
).agg({
    'NotPossible': 'sum'
})

possible_game = check_df.loc[check_df.NotPossible == 0]

In [71]:
def search_game_id(string):

    rule_search = re.compile(r'Game\s(.*)')

    number = rule_search.search(string)

    return int(number.group(1))

In [72]:
possible_game['ID'] = possible_game.Game.apply(search_game_id)

/tmp/ipykernel_603451/3433497914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  possible_game['ID'] = possible_game.Game.apply(search_game_id)


In [73]:
possible_game.ID.sum()

2416

# Part 2: Minimum sets to be possible

In [74]:
idx = input_data.groupby(['Game', 'Color'])['Number'].transform(max) == input_data['Number']

In [84]:
minimum_df = input_data[idx].drop('Set', axis=1)
minimum_df = minimum_df.drop_duplicates(subset=['Game', 'Color'], keep='first')

In [85]:
minimum_df = minimum_df.pivot_table(index='Game', columns='Color', values='Number')
minimum_df = minimum_df.assign(
    Result = lambda x: x.Blue * x.Red * x.Green
)

In [86]:
minimum_df.Result.sum()

63307